In [1]:
import urllib.request
import os.path
import numpy as np
import sys
sys.path.insert(0, './')
from utils import plot_mats

file_name = 'mnist_test_seq.npy'
url = 'http://www.cs.toronto.edu/~nitish/unsupervised_video/mnist_test_seq.npy'

if not os.path.isfile(file_name):
    print("could not find dataset: download it..")
    urllib.request.urlretrieve(url, file_name)
    print("download complete")

# Moving Mnist: 10.000 sequences of length 20 showing 2 digits moving in 64x64
moving_mnist = np.load(file_name) # shape: 20,10000,64,64
moving_mnist = np.rollaxis(moving_mnist, 1) # --> 10000,20,64,64

In [2]:
class ImageSplitter:
    """ splits the video set into shorts of n frames
    """
    def __init__(self, 
                 dataset,
                 n=3, 
                 ntest=256,
                 batchsize=64):
        """ ctor
            dataset: list of videos that have to be cut into n-grams
            n: integer, length of subsequence
            ntest: integer, number of items that belong to the test set
            batchsize: ~
            
        The data has to be arranged in following order:
            NUMBER_OF_VIDEOS, LENGTH_OF_VIDEO, HEIGHT, WIDTH
            
        """
        
        self.batchsize = batchsize
        self.batch_loop = 0
        self.n = n
        

        np.random.shuffle(dataset)
        
        self.test_set = dataset[0:ntest]
        self.train_set = dataset[ntest:]
        
        F, N, H, W = dataset.shape # Frames, Numbers, Height, Width
        
        
        self.vector_dimension = H * W
        self.same_batch_run = True
        
    def get_dimension(self):
        """ gets the data dimension
        """
        return self.vector_dimension

    def transform_to_n_gram(self, batch, ngram=None):
        """ transforms the batch into an n-gram (parameter n)
        batch: np.array((batchsize, 20, 64, 64))
        """
        if ngram is None:
            n = self.n
        else:
            n = ngram
        N, M, H, W = batch.shape
        seqs = M-n+1 # sequences per video
        
        Result = np.zeros((seqs * N, n, H, W))
        
        pos = 0
        for j in range(N):
            for i in range(seqs):
                Result[pos] = batch[j, i:i+n]
                pos += 1
        
        N, M, H, W = Result.shape
        return Result.reshape((N, M, H * W))
    
    def is_same_batch_run(self):
        if self.same_batch_run:
            return True
        else:
            self.same_batch_run = True
            return False
        
    
    def next_batch(self, ngram=None):
        """ returns the next batch
        """
        start = self.batch_loop
        end = self.batch_loop + self.batchsize
        N = self.train_set.shape[0]
        if N > end:
            self.same_batch_run = True
            self.batch_loop = end
            return self.transform_to_n_gram(
                self.train_set[start:end], ngram=ngram)
        else:
            self.same_batch_run = False
            diff = (N - start)
            end = self.batchsize - diff
            
            set1 = self.train_set[start:N]
            set2 = self.train_set[0:end]
            
            self.batch_loop = end
            return self.transform_to_n_gram(
                np.concatenate((set1, set2)), ngram=ngram)
    
    def get_test(self, ngram=None):
        """ returns the test data as ngram
        """
        return self.transform_to_n_gram(self.test_set, ngram=ngram)

# splitter = ImageSplitter(moving_mnist, n=3)
# batch = splitter.next_batch(ngram=2)
# print(batch[:,0,:].shape)

# test = splitter.get_test(ngram=2)
# print(test.shape)

# print(np.max(test), np.min(test))

In [4]:
import tensorflow as tf

# this function is 'borrowed' from keras
def random_binomial(shape, p=0.0, dtype=None, seed=None):
    """Returns a tensor with random binomial distribution of values.
    # Arguments
        shape: A tuple of integers, the shape of tensor to create.
        p: A float, `0. <= p <= 1`, probability of binomial distribution.
        dtype: String, dtype of returned tensor.
        seed: Integer, random seed.
    # Returns
        A tensor.
    """
    if dtype is None:
        dtype = 'float32'
    if seed is None:
        seed = np.random.randint(10e6)
    return tf.where(tf.random_uniform(shape, dtype=dtype, seed=seed) <= p,
                    tf.ones(shape, dtype=dtype),
                    tf.zeros(shape, dtype=dtype))


class PredictiveGatingPyramid:
    """ implementation of the pgp
    """
    
    def __init__(self, 
                 depth=2, 
                 numfilters=512, 
                 numHidden=256,
                 learning_rate=0.01,
                 normalize_data=True):
        assert depth > 0
        assert depth == 2, "Other depth than 2 not supported"
        self.depth = depth
        self.F = numfilters
        self.H = numHidden
        self.learningRate = learning_rate
        self.is_trained = False
        self.normalize_data = normalize_data
    
    
    def train(self, X, epochs=100, pre_epochs=100, print_debug=True):
        """ trains the model
        
            X: training data: must be organized as follows:
                Number_of_videos, video_length, H, W
            epochs: number of epochs to run for final training
            pre_epochs: number of epochs for training initial layer
        """
      
        X = X.astype('float32')  # hopefully we don't run OOMem here..
        
        if self.normalize_data:
            X -= X.mean(0)[None, :]
            X /= X.std(0)[None, :] + X.std() * 0.1

        splitter = ImageSplitter(X, n=self.depth+1)

        F = self.F
        H = self.H
        lr = self.learningRate
        dim = splitter.get_dimension()
        numpy_rng = np.random.RandomState(1)
        
        # pretrain the early layer for faster convergence
        
        x = tf.placeholder(tf.float32, [None, dim])
        y = tf.placeholder(tf.float32, [None, dim])
        
        U1 = tf.Variable(tf.random_normal(shape=(dim, F)) * 0.01)
        V1 = tf.Variable(tf.random_normal(shape=(dim, F)) * 0.01)
        W1 = tf.Variable(
            numpy_rng.uniform(low=-0.01, high=+0.01, 
                              size=( F, H)).astype('float32'))

        b_U1 = tf.Variable(np.zeros(F, dtype='float32'))
        b_V1 = tf.Variable(np.zeros(F, dtype='float32'))
        b_W1 = tf.Variable(np.zeros(H, dtype='float32'))
        b_U1_out = tf.Variable(np.zeros(dim, dtype='float32'))
        b_V1_out = tf.Variable(np.zeros(dim, dtype='float32'))
        b_W1_out = tf.Variable(np.zeros(F, dtype='float32'))
        
        
        m1 = tf.sigmoid(tf.matmul(tf.multiply(
            tf.matmul(x,U1) + b_U1,
            tf.matmul(y,V1) + b_V1), W1) + b_W1)

        ox = tf.matmul(tf.multiply(
                tf.matmul(m1,tf.transpose(W1)) + b_W1_out,
                tf.matmul(y,V1) + b_V1),tf.transpose(U1))+ b_U1_out
        oy = tf.matmul(tf.multiply(
                tf.matmul(m1,tf.transpose(W1)) + b_W1_out,
                tf.matmul(x,U1) + b_U1), 
            tf.transpose(V1)) + b_V1_out

        cost_1 = tf.nn.l2_loss(ox-x) + tf.nn.l2_loss(oy-y)
        optimizer_1 = tf.train.AdamOptimizer(learning_rate=0.0001)\
            .minimize(cost_1)
            
        norm_U1 = tf.nn.l2_normalize(U1, [0,1], epsilon=1e-12, name=None)
        norm_V1 = tf.nn.l2_normalize(V1, [0,1], epsilon=1e-12, name=None)

        normalize_U1 = U1.assign(norm_U1)
        normalize_V1 = V1.assign(norm_V1)
        
            
        with tf.Session() as sess:
            init = tf.global_variables_initializer()
            sess.run(init)
            
            test_set = splitter.get_test(ngram=2)
            X_ = test_set[:,0,:]
            Y_ = test_set[:,1,:]
            n = test_set.shape[0]
            for epoch in range(pre_epochs):
                while splitter.is_same_batch_run():
                    batch = splitter.next_batch(ngram=2)
                    
                
                cost_ = sess.run(cost_1, feed_dict={x: X_, y: Y_}) / n
                if print_debug:
                    print ("Pretrain: Epoch: %03d/%03d cost: %.9f" %\
                               (epoch,pre_epochs ,cost_) )
            
        
        print("train end")

print('xx')

model = PredictiveGatingPyramid(depth=2)
model.train(moving_mnist)

xx
Pretrain: Epoch: 000/100 cost: 2496.105263158
Pretrain: Epoch: 001/100 cost: 2496.105674342
Pretrain: Epoch: 002/100 cost: 2496.104851974
Pretrain: Epoch: 003/100 cost: 2496.105263158
Pretrain: Epoch: 004/100 cost: 2496.105468750
Pretrain: Epoch: 005/100 cost: 2496.104029605
Pretrain: Epoch: 006/100 cost: 2496.104440789


KeyboardInterrupt: 